In [1]:
import pprint
from functools import partial
import baostock as bs
import pandas as pd
from gluonts.dataset.common import ListDataset
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer
from pathlib import Path
from gluonts.model.predictor import Predictor


In [27]:
def mygetstockdata(code):
    
   # print('login respond error_code:'+lg.error_code)
   # print('login respond  error_msg:'+lg.error_msg)
    rs = bs.query_history_k_data_plus(code,
        "date,close,volume,turn",
        start_date='2019-01-01', 
        frequency="d", adjustflag="2") #frequency="d"取日k线，adjustflag="3"默认不复权
   
    return rs.get_data()

In [28]:
#dd = mygetstockdata('sz.300455')
#print( list( dd.close))

In [29]:
import json
liststock =['sz.300807','sz.300789','sz.300771','sz.300546','sz.300479','sz.300462','sz.300455','sz.300449','sz.300386','sz.300368']
#liststock = ['sz.300462']
listdic = []
lg = bs.login()
for ite in liststock:
    dd = mygetstockdata(ite)
    dic = {"start":dd.date[0],"target":list(dd.close),"cat":int(liststock[0].split('.')[1]),"dynamic_feat":[list(dd.volume),list(dd.turn)]}
    #strjon = json.dumps(dic)
    listdic.append(dic)

bs.logout()

login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!
login success!
logout success!


In [31]:
#jjon = json.dumps(listdic)
#print(jjon)



[{"start": "2019-12-19", "target": ["25.0156720800", "27.5211694800", "30.2723038800", "33.2985517200", "36.6293894400", "40.2942934800", "44.3227402800", "48.7540317600", "53.6274698400", "58.9921819200", "64.8874699200", "61.8022692000", "65.8405414800", "64.1800353600", "62.5391802000", "59.2279934400", "58.1275396800", "58.2847473600", "56.6930196000", "56.8404018000", "55.4157072000", "56.6438922000", "54.4429846800", "52.6940492400", "49.2256548000", "44.3030893200", "41.8368938400", "43.0159514400", "44.6469811200", "45.8260387200", "46.3074872400", "45.2561608800", "45.9832464000", "48.2038048800", "51.1907508000", "52.2224262000", "52.7923040400", "51.0630195600", "53.5586914800", "53.1263703600", "52.8905588400", "51.2300527200", "51.3184820400", "49.1274000000", "44.2146600000", "46.8871905600", "47.1033511200", "48.1448520000", "48.3413616000", "49.3730370000", "50.5717455600", "50.8861609200", "55.9757595600", "58.0685868000", "52.5859689600", "51.7802796000", "48.64595148

In [30]:
jjon = json.dumps(listdic)
traindata = ListDataset(
    listdic,
    freq = "1d"
)

#print(traindata.list_data )

[{'start': '2019-12-19', 'target': ['25.0156720800', '27.5211694800', '30.2723038800', '33.2985517200', '36.6293894400', '40.2942934800', '44.3227402800', '48.7540317600', '53.6274698400', '58.9921819200', '64.8874699200', '61.8022692000', '65.8405414800', '64.1800353600', '62.5391802000', '59.2279934400', '58.1275396800', '58.2847473600', '56.6930196000', '56.8404018000', '55.4157072000', '56.6438922000', '54.4429846800', '52.6940492400', '49.2256548000', '44.3030893200', '41.8368938400', '43.0159514400', '44.6469811200', '45.8260387200', '46.3074872400', '45.2561608800', '45.9832464000', '48.2038048800', '51.1907508000', '52.2224262000', '52.7923040400', '51.0630195600', '53.5586914800', '53.1263703600', '52.8905588400', '51.2300527200', '51.3184820400', '49.1274000000', '44.2146600000', '46.8871905600', '47.1033511200', '48.1448520000', '48.3413616000', '49.3730370000', '50.5717455600', '50.8861609200', '55.9757595600', '58.0685868000', '52.5859689600', '51.7802796000', '48.64595148

In [12]:


prediction_length = 30
estimator = DeepAREstimator(
    prediction_length=prediction_length,
    context_length=30,
    freq="1d",
    trainer=Trainer(ctx="cpu",
                    epochs=30,
                    learning_rate=1e-3,
                    num_batches_per_epoch=500
                   )
)
train_output = estimator.train(traindata)

train_output.serialize(Path("./tmp/"))


  0%|          | 0/100 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


 53%|█████▎    | 53/100 [00:05<00:04,  9.85it/s, epoch=1/5, avg_epoch_loss=2.68]


KeyboardInterrupt: 

NameError: name 'transformation' is not defined